In [3]:
import gdal

In [11]:
filepath = r"../data/raw/satw/temp.tiff"
filepath2 = r"../data/raw/satw/temp.tiff"


# Open the file:
raster = gdal.Open(filepath)

# Check type of the variable 'raster'
type(raster)

osgeo.gdal.Dataset

In [12]:
# Projection
print(raster.GetProjection())

# Dimensions
print( raster.RasterXSize )
print( raster.RasterYSize )

# Number of bands
print( raster.RasterCount )

# Metadata for the raster dataset
raster.GetMetadata()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
560
886
3


{'AREA_OR_POINT': 'Area',
 'TIFFTAG_RESOLUTIONUNIT': '2 (pixels/inch)',
 'TIFFTAG_XRESOLUTION': '72',
 'TIFFTAG_YRESOLUTION': '72'}

#### Read Raster as Array

In [25]:
# Read raster data as numeric array from GDAL Dataset
rasterArray = raster.ReadAsArray()

In [26]:
type(rasterArray)


numpy.ndarray

#### Investigate Bands

In [23]:
# Read the raster band as separate variable
band = raster.GetRasterBand(1)

# Check type of the variable 'band'
print( type(band))

# Data type of the values
gdal.GetDataTypeName(band.DataType)

<class 'osgeo.gdal.Band'>


'Byte'

In [24]:
# # Compute statistics if needed
# if band.GetMinimum() is None or band.GetMaximum()is None:
#     band.ComputeStatistics(0)
#     print("Statistics computed.")

# # Fetch metadata for the band
# band.GetMetadata()

# # Print only selected metadata:
# print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none
print ("[ MIN ] = ", band.GetMinimum())
print ("[ MAX ] = ", band.GetMaximum())

[ MIN ] =  None
[ MAX ] =  None
